In [8]:
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
import random
import time
import cvxpy as cp
import pandas as pd
from sklearn.model_selection import train_test_split
import collections

In [9]:
TRAIN_DATA = pd.read_csv('transfer_train-1.csv')
TEST_DATA=pd.read_csv('transfer_test-1.csv')

TRAIN_DATA_W_LABELS = TRAIN_DATA[(TRAIN_DATA['Digit']==1) | (TRAIN_DATA['Digit']==7)]

D = 8 

ONLY_LABELS = TRAIN_DATA_W_LABELS[['Digit']]


ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 7, 'Digit'] = -1


TRAIN_DATA_NO_LABEL = TRAIN_DATA_W_LABELS.drop('Digit', axis=1)

Train_data_normalized=TRAIN_DATA_NO_LABEL.copy()

for column in Train_data_normalized.columns:
    Train_data_normalized[column] = (Train_data_normalized[column] - Train_data_normalized[column].min()) / (Train_data_normalized[column].max() - Train_data_normalized[column].min()) 

TEST_DATA_W_LABELS = TEST_DATA[(TEST_DATA['Digit']==1) | (TEST_DATA['Digit']==7)]

ONLY_LABELS_TEST = TEST_DATA_W_LABELS[['Digit']]

ONLY_LABELS_TEST.loc[ONLY_LABELS_TEST['Digit'] == 7, 'Digit'] = -1

TEST_DATA_NO_LABEL = TEST_DATA_W_LABELS.drop('Digit', axis=1)

Test_data_normalized=TEST_DATA_NO_LABEL.copy()

for column in Test_data_normalized.columns:
    Test_data_normalized[column] = (Test_data_normalized[column] - Test_data_normalized[column].min()) / (Test_data_normalized[column].max() - Test_data_normalized[column].min())  

/tmp/ipykernel_384440/1232886161.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 7, 'Digit'] = -1
/tmp/ipykernel_384440/1232886161.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ONLY_LABELS_TEST.loc[ONLY_LABELS_TEST['Digit'] == 7, 'Digit'] = -1


In [10]:
X_train_np=Train_data_normalized.to_numpy()
print(X_train_np)
print(X_train_np.shape)
y_train_np=ONLY_LABELS.to_numpy()
print(y_train_np.shape)
X_test=Test_data_normalized.to_numpy()
y_test=ONLY_LABELS_TEST.to_numpy()

[[0.58461538 0.68627451 1.         1.         0.86206897 0.75
  0.86170213 0.68493151]
 [0.2        0.56862745 0.13235294 0.36       0.09195402 0.
  0.         0.04109589]
 [0.10769231 0.66666667 0.69117647 1.         1.         0.98
  0.86170213 0.90410959]
 [0.70769231 1.         1.         0.95       0.7816092  0.7
  0.60638298 0.61643836]
 [0.2        0.         0.64705882 0.73       1.         1.
  1.         1.        ]
 [0.84615385 0.80392157 0.85294118 0.48       0.72413793 0.06
  0.58510638 0.        ]
 [1.         0.58823529 0.75       0.44       0.62068966 0.08
  0.4787234  0.        ]
 [0.46153846 1.         1.         1.         0.6091954  0.76
  0.5        0.67123288]
 [0.13846154 0.19607843 0.10294118 0.2        0.18390805 0.
  0.         0.42465753]
 [0.         0.03921569 0.         0.         0.         0.13
  0.23404255 0.43835616]]
(10, 8)
(10, 1)


In [11]:
train_data_qty = len(X_train_np)
print(train_data_qty)

10


In [14]:
C = 2

w_source = np.array([[0.00330738], [0.0311968], [ 0.02214532], [ 0.01202985], [ 0.00929693], [-0.02798008], [-0.02736712], [ 0.01508687]])

w = cp.Variable((D, 1))

b = cp.Variable()

epsilon = cp.Variable((train_data_qty, 1))

objective = cp.Minimize(cp.sum(cp.square(w))*0.5 + cp.sum(cp.square(epsilon)*C))

constraints = [cp.multiply(y_train_np, (X_train_np @ (w + w_source) + b)) >= 1 - epsilon,  epsilon >= 0]

prob = cp.Problem(objective, constraints)
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var w = {}, b = {}".format(w.value, b.value))

status: optimal
optimal value 11.841898979688638
optimal var w = [[-0.4870198 ]
 [ 0.06900336]
 [-0.89219711]
 [ 1.09496165]
 [ 0.48455474]
 [ 0.64941973]
 [ 0.34298911]
 [-0.24788309]], b = -0.6044829714703345


In [15]:
eps = 1e-2 # select support vectors
random.seed(1) # For reproducibility

sup = y_train_np*(X_train_np.dot(w.value + w_source)+b.value)-1 


sup_v1 = ((-eps<sup) & (sup<eps)).flatten()

collections.Counter(sup_v1)

Xt = X_test

#Xt.shape

yt = y_test

#yt.shape

predictions = Xt.dot(w.value + w_source) + b.value

predictions[predictions<0] = -1

predictions[predictions>=0] = 1

count_wrong_predictions = 0
for i in range(len(predictions)):
    if (yt[i] != predictions[i]):
        count_wrong_predictions+=1
        
#count_wrong_predictions

correct_predictions = (len(predictions) - count_wrong_predictions)
#correct_predictions

accuracy = correct_predictions/len(predictions)

print("Accuracy =",accuracy*100,"%")

Accuracy = 76.92307692307693 %
